In [2]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

# TAMBAHAN
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip list

In [4]:
def read_db():
  db = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/VERIFY/dB_tbh.csv')

  judul = db['Judul'].apply(str).values.tolist()
  isi = db['Isi berita'].apply(str).values.tolist()   # DIGANTI JADI ISI BERITA
  url = db['Link'].values.tolist()
  date = db['Tanggal'].values.tolist()

  return judul, isi, url, date

In [5]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [6]:
def cek_kesamaan(headline_input, pembanding):
  cosine_sim = []
  for p in pembanding:
    cs = get_cosine_sim(headline_input, p).min()
    cosine_sim.append(cs)

  maks = cosine_sim.index(max(cosine_sim))
  top = pembanding[maks]

  return max(cosine_sim), top

In [7]:
def write_json(judul, url, date, kategori, penjelasan, preview):
  to_json = {
      'code': 200,
      'judul': judul,
      'url': url,
      'tanggal': date,
      'kategori': kategori,             # TAMBAHAN
      'penjelasan': penjelasan,         # TAMBAHAN
      'preview': preview                # TAMBAHAN
  }
  with open("message.json", "w") as f:
      json.dump(to_json, f)

def write_json_notfound():
  to_json = {
      'code': 404,
      'isi': 'Tidak ditemukan berita'
  }
  with open("message.json", "w") as f:
      json.dump(to_json, f)

In [13]:
# TAMBAHAN

def filter_content(content):
  content = content.replace('\n', ' ')
  regex = r"(.*)(Kategori:.*)(===== Sumber:.*)(===== Penjelasan:.*)(===== Referensi:.*)"
  matches = re.match(regex, content, re.M|re.I)

  if matches == None:
    kategori = None
    penjelasan = content 
  else:  
    kategori = matches.group(2).replace('Kategori:', '')
    penjelasan = matches.group(4).replace('===== Penjelasan:', '')
  
  preview = ' '.join(nltk.tokenize.sent_tokenize(penjelasan)[:2])

  return kategori, penjelasan, preview

In [9]:
def search_from_db(headline_input):
  judul, isi, url, date = read_db()

  title_similarity_score, selected_title = cek_kesamaan(headline_input, judul)
  
  # Treshold apakah mencari kesamaan dalam isi berita diperlukan?
  # kondisi ketika pencarian tidak dilakukan
  if title_similarity_score > 0:
    idx = judul.index(selected_title)
    kategori, penjelasan, preview = filter_content(isi[idx])                          # BUAT MISAHIN
    write_json(selected_title, url[idx], date[idx], kategori, penjelasan, preview)    # EDIT
    print('berita tersedia di judul')
  
  # kondisi ketika pencarian perlu dilakukan
  else:
    content_similarity_score, selected_content = cek_kesamaan(headline_input, isi)

    # kondisi ketika tidak ditemukan kecocokan sama sekali
    if content_similarity_score == 0:
      write_json_notfound()
      print('berita belum tersedia')
    else:
      idx = isi.index(selected_content)
      kategori, penjelasan, preview = filter_content(selected_content)               # BUAT MISAHIN
      write_json(judul[idx], url[idx], date[idx], kategori, penjelasan, preview)     # EDIT
      print('berita tersedia di isi')

In [15]:
search_from_db('Dieffebachia , Tanaman hias yang beracun')

berita tersedia di judul


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
